In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
from utils import IntraSensorDataset, get_train_validation_split
from torch.utils.data import DataLoader
from models import DualModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve
import numpy as np

In [ ]:
validation_split = 0.1
batch_size = 32
num_workers = 0
pin_memory = False
lr = 0.001

num_epochs = 50

train_dir = './LivDet/2013/Training/BiometrikaTrain'
test_dir = './LivDet/2013/Testing/BiometrikaTest'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.RandomAffine(
        degrees=(-20, 20),          # Rotation
        translate=(0.2, 0.2),       # Horizontal/vertical shift
        shear=(-20, 20),            # Shear
        scale=(0.8, 1.2),           # Zoom
        interpolation=InterpolationMode.NEAREST,
        fill=0
    ),
    T.RandomHorizontalFlip(p=0.5),  # Horizontal flip
    T.RandomVerticalFlip(p=0.5)     # Vertical flip
])

test_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

In [6]:
train_dataset = IntraSensorDataset(data_dir=train_dir, transform=train_transform)
train_set, val_set = get_train_validation_split(dataset=train_dataset, validation_split=validation_split)
test_set = IntraSensorDataset(data_dir=test_dir, transform=test_transform)

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

In [7]:
model = DualModel(num_classes=1).to(device)
model = nn.DataParallel(model)

In [8]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        
        outputs = model(data)
        loss = criterion(outputs.squeeze(), targets)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        if batch_idx % 50 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}]   Batch Index [{batch_idx+1}/{len(train_loader)}]   Train Loss: {loss:.4f}')

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    val_running_loss = 0.0
    
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device, dtype=torch.float)
            
            outputs = model(data)
            loss = criterion(outputs.squeeze(), targets)
            val_running_loss += loss.item()
    
    avg_val_loss = val_running_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}]   Train Loss: {avg_train_loss:.4f}   Val Loss: {avg_val_loss:.4f}')

In [ ]:
# Plot losses
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.set_title('Model Loss')
plt.set_xlabel('Epoch')
plt.set_ylabel('Loss')
plt.legend()
plt.show()

In [9]:
# Testing phase
model.eval()

all_targets = []
all_probabilities = []

with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device, dtype=torch.float)
        
        outputs = model(data)
        probabilities = torch.sigmoid(outputs.squeeze())
        
        all_targets.extend(targets.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

targets = np.array(all_targets).astype(int)
probabilities = np.array(all_probabilities)

In [10]:
fpr, tpr, thresholds = roc_curve(targets, probabilities)
j_scores = tpr - fpr
optimal_idx = np.argmax(j_scores)
optimal_threshold = thresholds[optimal_idx]

predictions = (probabilities >= optimal_threshold).astype(int)

In [11]:
genuine_mask = (targets == 0)  # Bona fide samples
attack_mask = (targets == 1)   # Attack samples

genuine_count = np.sum(genuine_mask)
attack_count = np.sum(attack_mask)

# Calculate errors
# APCER: False Accept Rate (attack samples incorrectly classified as genuine)
genuine_as_attack = predictions[genuine_mask]  # What the model predicted for genuine samples
apcer = np.sum(genuine_as_attack) / genuine_count if genuine_count > 0 else 0  # FP / Total Genuine

# BPCER: False Reject Rate (genuine samples incorrectly classified as attack)
attack_as_genuine = 1 - predictions[attack_mask]  # What the model predicted as genuine for attack samples
bpcer = np.sum(attack_as_genuine) / attack_count if attack_count > 0 else 0  # FN / Total Attack

# Also calculate ACER (Average Classification Error Rate)
acer = (apcer + bpcer) / 2

acc = 1 - acer

accuracy = accuracy_score(targets, predictions) * 100

print(f"APCER: {apcer*100:.2f}%")
print(f"BPCER: {bpcer*100:.2f}%")
print(f"ACER:  {acer*100:.2f}%")
print(f"ACC*:  {acc*100:.2f}%")
print(f"ACC:   {accuracy:.2f}%")

APCER: 26.80%
BPCER: 62.30%
ACER:  44.55%
ACC*:  55.45%
ACC:   55.45%
